In [160]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import os

copy of notebook 02 to chnange the marketing logic, also datasetes now have the 2 marketing columns instead of the old mktgparams one

In [161]:
# sc_after_march (calendarevent logs) 
file_path = r"C:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\new data\sc_from_march.csv"

df1 = pd.read_csv(file_path)
df1.columns = df1.columns.str.lower()

#sc_ older contracts (calendarevent)
path = r"C:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\new data\sc_older_contracts.csv"
df2 = pd.read_csv(path)
df2.columns = df2.columns.str.lower()

#concat
combined_df = pd.concat([df1, df2], ignore_index=True)
df = combined_df.copy()

# qualitative (slider + contract)

file2 = r"C:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\new data\info_client2.csv"

df3 = pd.read_csv(file2)
df3.columns = df3.columns.str.lower()

In [162]:
behaviour = pd.read_csv(r"processed_data/df_model.csv") # manualy saved leadtime info. hafter having gotten modified in older attempts/notebook
behaviour = behaviour.drop(['converted', 'lead_to_sc1_days', 'sc1_schedule_to_appointment_days', 'showed_up_sc1' ], axis=1)

In [163]:

behaviour.columns = behaviour.columns.str.lower()
behaviour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19981 entries, 0 to 19980
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          19981 non-null  int64  
 1   total_bc_attempts           19981 non-null  int64  
 2   total_bc_outcomes           19981 non-null  int64  
 3   lead_to_first_bc_days       19981 non-null  float64
 4   bc_duration_days            19981 non-null  float64
 5   bc_frequency                19981 non-null  float64
 6   positive_outcomes_count     19981 non-null  int64  
 7   negative_outcomes_count     19981 non-null  int64  
 8   noshow_outcomes_count       19981 non-null  int64  
 9   positive_outcome_ratio      19981 non-null  float64
 10  negative_outcome_ratio      19981 non-null  float64
 11  noshow_outcome_ratio        19981 non-null  float64
 12  reachability_score          19981 non-null  float64
 13  outcome_trend               199

In [164]:
#AD HOC
# trying to match the positive contracts to leadtime
#try1 = df2.merge(behaviour, left_on='leadid', right_on='id', how='left', indicator=True)
#df2.info()
#print(df2["leadid"].nunique())
#try1.info()
#print(try1["id"].nunique())
#print(behaviour["id"].nunique())

In [165]:
# 83346 sc logs of 28193 unique requests
combined_df.info()
combined_df["requestid"].nunique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83407 entries, 0 to 83406
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   createdat  83407 non-null  object
 1   deletedat  34985 non-null  object
 2   requestid  83407 non-null  int64 
 3   tsstart    83407 non-null  object
 4   type       83407 non-null  object
 5   is_passed  83407 non-null  int64 
 6   leadid     83407 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 4.5+ MB


24520

In [166]:
# behavioral data that has 2.7k positive examples
df3.info()
df3["grosscontractsigned"].value_counts()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126489 entries, 0 to 126488
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   requestid               126489 non-null  int64  
 1   zipregion               123407 non-null  object 
 2   evaluationtime          119790 non-null  object 
 3   desiredinstallationend  119870 non-null  object 
 4   electricitybill         105858 non-null  float64
 5   heatingbill             92462 non-null   float64
 6   channel__campaign       126489 non-null  object 
 7   channel                 125384 non-null  object 
 8   grosscontractsigned     2795 non-null    float64
 9   selfipaimportedat       2632 non-null    object 
dtypes: float64(3), int64(1), object(6)
memory usage: 9.7+ MB


grosscontractsigned
1.0    2746
0.0      49
Name: count, dtype: int64

In [167]:
# drop duplicates in df3 -info client-,they were found in notebook 01. probably because same requestid have 2 entries in airtable contracts, one 0 one 1. we wanna keep only 1s.

# Step 1: Count requestids that have duplicates
duplicate_requestids = df3[df3['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Number of requestids with duplicates: {len(duplicate_requestids)}")
print(f"Total duplicate rows: {df3['requestid'].duplicated().sum()}")

# Step 2: Drop absolute duplicates (excluding leadid column)
comparison_cols = [col for col in df3.columns if col != 'leadid']
df3_before = len(df3)
df3 = df3.drop_duplicates(subset=comparison_cols, keep='first')
df3_after = len(df3)

print(f"\nDropped {df3_before - df3_after} absolute duplicate rows")

# Step 3: From the remaining duplicate requestids, drop rows where grosscontractsigned = 0
remaining_duplicate_requestids = df3[df3['requestid'].duplicated(keep=False)]['requestid'].unique()

if len(remaining_duplicate_requestids) > 0:
    print(f"Remaining duplicate requestids before grosscontractsigned cleanup: {len(remaining_duplicate_requestids)}")
    
    # Create condition: either NOT a duplicate requestid, OR grosscontractsigned != 0
    condition = (~df3['requestid'].isin(remaining_duplicate_requestids)) | (df3['grosscontractsigned'] != 0)
    
    df3_before_net_drop = len(df3)
    df3 = df3[condition]
    df3_after_net_drop = len(df3)
    
    print(f"Dropped {df3_before_net_drop - df3_after_net_drop} rows where grosscontractsigned = 0 from duplicate requestids")

# Step 4: Final duplicate check
print("\n" + "="*50)
print("FINAL DUPLICATE CHECK")
print("="*50)

final_duplicate_requestids = df3[df3['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Number of requestids with duplicates: {len(final_duplicate_requestids)}")
print(f"Total duplicate rows: {df3['requestid'].duplicated().sum()}")

if len(final_duplicate_requestids) > 0:
    # Check what columns still differ
    all_differing_columns = set()
    for requestid in final_duplicate_requestids:
        group = df3[df3['requestid'] == requestid]
        for col in df3.columns:
            if col != 'requestid' and group[col].nunique() > 1:
                all_differing_columns.add(col)
    
    print(f"Differing columns: {sorted(all_differing_columns)}")
else:
    print(f"✅ No remaining duplicates!")

Number of requestids with duplicates: 52
Total duplicate rows: 52

Dropped 5 absolute duplicate rows
Remaining duplicate requestids before grosscontractsigned cleanup: 47
Dropped 47 rows where grosscontractsigned = 0 from duplicate requestids

FINAL DUPLICATE CHECK
Number of requestids with duplicates: 0
Total duplicate rows: 0
✅ No remaining duplicates!


In [168]:
df3["grosscontractsigned"].value_counts()

grosscontractsigned
1.0    2741
0.0       2
Name: count, dtype: int64

#### Create Derived Features

In [169]:
# fixing appointment types

type_mapping = {
    'FU': 'FU',
    'FUVD': 'FU', 
    'REMINDER': 'FU',
    'VDFIELD': 'SC',
    'VD': 'SC'
}

# Create grouped_type column
df['grouped_type'] = df['type'].map(type_mapping)

In [170]:
# Create condition for net events (not cancelled and past)
df['is_net_event'] = df['deletedat'].isna() & df['is_passed']

In [171]:
# Gross counts (all events)
gross_counts = df.groupby(['requestid', 'grouped_type']).size().unstack(fill_value=0)
gross_counts.columns = [f'gross_{col}' for col in gross_counts.columns]

# Net counts (only events that actually happened)
net_counts = df[df['is_net_event']].groupby(['requestid', 'grouped_type']).size().unstack(fill_value=0)
net_counts.columns = [f'net_{col}' for col in net_counts.columns]

In [172]:
# Combine gross and net counts
counts_df = gross_counts.join(net_counts, how='outer').fillna(0)

In [173]:
counts_df.index.duplicated().any()

np.False_

In [174]:
# turn self ipa event to bbinary 
df3['selfipa_done'] = df3['selfipaimportedat'].notnull().astype(int)
df3.drop('selfipaimportedat', axis=1, inplace=True) 

In [175]:
#  handle marketing
# new marketing source


def create_aggregated_column(df):
    """
    Creates the 'Aggregated' column based on Channel and CHANNEL__CAMPAIGN values
    """
    def get_aggregated_value(row):
        # Check Youtube first
        if row.get('Youtube') == 1:
            return "Youtube"
        
        # Check specific CHANNEL__CAMPAIGN values
        channel_campaign = row.get('channel_campaign')
        if channel_campaign == "d2d":
            return "d2d"
        elif channel_campaign == "form_classico":
            return "Form_FB"
        elif channel_campaign == "ranger":
            return "ranger"
        elif channel_campaign == "referral_link":
            return "Referral"
        elif channel_campaign == "referral_manual":
            return "Referral"
        
        # Check Channel column (case insensitive)
        channel = row.get('channel')
        if channel and pd.notna(channel):
            channel_lower = channel.lower()
            
            if channel_lower == "affiliation":
                return "Affiliation"
            elif channel_lower == "outbrain":
                return "Outbrain"
            elif channel_lower in ["taboola", "taboola_it"]:
                return "Taboola"
            elif channel_lower in ["facebook", "fb", "ig", "instagram", "facebook_marketplace"]:
                return "Meta"
            elif channel_lower in ["search ads", "searchads", "google", "google ads"]:
                return "Google"
            elif channel_lower in ["organic_brand", "organic", "direct"]:
                return "Organic"
            elif channel_lower == "seo":
                return "SEO"
            elif channel_lower == "criteo":
                return "Criteo"
            elif channel_lower == "mediago":
                return "MediaGo"
            elif channel_lower == "tiktok":
                return "Tik Tok"
        
        return "Other"
    
    df['aggregated'] = df.apply(get_aggregated_value, axis=1)
    df.drop('channel', axis=1, inplace=True)
    df.drop('channel__campaign', axis=1, inplace=True)
    return df

In [176]:
df3 =   create_aggregated_column(df3)

In [177]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 126437 entries, 0 to 126488
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   requestid               126437 non-null  int64  
 1   zipregion               123357 non-null  object 
 2   evaluationtime          119738 non-null  object 
 3   desiredinstallationend  119818 non-null  object 
 4   electricitybill         105809 non-null  float64
 5   heatingbill             92430 non-null   float64
 6   grosscontractsigned     2743 non-null    float64
 7   selfipa_done            126437 non-null  int64  
 8   aggregated              126437 non-null  object 
dtypes: float64(3), int64(2), object(4)
memory usage: 9.6+ MB


In [178]:
df3["aggregated"].value_counts()

aggregated
Meta           41212
Outbrain       27213
Google         24548
Other           9295
Organic         7771
Taboola         5172
Affiliation     3987
Tik Tok         3799
MediaGo         1957
Criteo          1306
SEO              177
Name: count, dtype: int64

In [179]:
# Step 1: Convert createdat to datetime
df['createdat'] = pd.to_datetime(df['createdat'])

# Step 2: Get first gross SC and first net FU timestamps
first_gross_sc = df[df['grouped_type'] == 'SC'].groupby('requestid')['createdat'].min()
first_net_fu = df[(df['grouped_type'] == 'FU') & df['is_net_event']].groupby('requestid')['createdat'].min()

# Step 3: Calculate time difference in hours
counts_df['time_first_sc_to_first_net_fu'] = (first_net_fu - first_gross_sc).dt.total_seconds() / 3600

In [180]:
# drop older requests that havent had a sc since march
counts_df = counts_df[counts_df['gross_SC'] != 0]
counts_df['gross_SC'].value_counts() 

gross_SC
1     15378
2      3169
3       736
4       230
5        51
6        16
7         6
10        2
8         1
23        1
Name: count, dtype: int64

In [181]:
# handle missing value for customers who never made to fu
counts_df['time_first_sc_to_first_net_fu'] = counts_df['time_first_sc_to_first_net_fu'].fillna(-1)

# Check final dataset
print("Final dataset info:")
print(f"Shape: {counts_df.shape}")
print(f"No missing values: {counts_df.isnull().sum().sum() == 0}")

Final dataset info:
Shape: (19590, 5)
No missing values: True


In [182]:
# problematic entries with negative time difference: 
# Separate real negative times from NaN fill values
real_negative = counts_df[
    (counts_df['time_first_sc_to_first_net_fu'] < 0) & 
    (counts_df['time_first_sc_to_first_net_fu'] != -1)
]

actual_valid_times = counts_df[counts_df['time_first_sc_to_first_net_fu'] >= 0]

print(f"Actual negative time differences (not NaN fills): {len(real_negative)}")
print(f"Positive time differences: {len(actual_valid_times)}")
print(f"NaN fills (-1): {(counts_df['time_first_sc_to_first_net_fu'] == -1).sum()}")

if len(real_negative) > 0:
    print(f"\nReal negative values range: {real_negative['time_first_sc_to_first_net_fu'].min():.2f} to {real_negative['time_first_sc_to_first_net_fu'].max():.2f} hours")


Actual negative time differences (not NaN fills): 17
Positive time differences: 8576
NaN fills (-1): 10997

Real negative values range: -3764.75 to -22.24 hours


In [183]:
#handle negative time difference
# Remove outliers entirely (set to NaN, then fill with -1)
counts_df.loc[counts_df['time_first_sc_to_first_net_fu'] < -1, 'time_first_sc_to_first_net_fu'] = -1

In [184]:
# Verify the cleanup
print("After removing outliers:")
print(f"Negative values (excluding -1 fills): {((counts_df['time_first_sc_to_first_net_fu'] < 0) & (counts_df['time_first_sc_to_first_net_fu'] != -1)).sum()}")
print(f"Valid positive time differences: {(counts_df['time_first_sc_to_first_net_fu'] > 0).sum()}")
print(f"NaN fills (-1): {(counts_df['time_first_sc_to_first_net_fu'] == -1).sum()}")

# Final dataset summary
print(f"\nFinal dataset shape: {counts_df.shape}")
print("Ready to join with target variable!")

After removing outliers:
Negative values (excluding -1 fills): 0
Valid positive time differences: 8537
NaN fills (-1): 11014

Final dataset shape: (19590, 5)
Ready to join with target variable!


In [185]:
# at this point requestid is the index. make it explicit


counts_df = counts_df.reset_index()
counts_df.head()


,requestid,gross_FU,gross_SC,net_FU,net_SC,time_first_sc_to_first_net_fu
0,1191,0,1,0.0,1.0,-1.000000
1,1271,0,1,0.0,1.0,-1.000000
2,1285,1,1,1.0,1.0,0.000000
3,1294,0,1,0.0,1.0,-1.000000
4,1299,2,1,2.0,1.0,0.000278


In [186]:
# lets see net_sc accuracy
mask1 = counts_df[counts_df['gross_SC'] == 0]
mask1
# 7k didnt make it to net sc but has grosssc

,requestid,gross_FU,gross_SC,net_FU,net_SC,time_first_sc_to_first_net_fu


In [187]:
counts_df.info()
#leadid not present here, we moved to 19.5 k distinct requests 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19590 entries, 0 to 19589
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   requestid                      19590 non-null  int64  
 1   gross_FU                       19590 non-null  int64  
 2   gross_SC                       19590 non-null  int64  
 3   net_FU                         19590 non-null  float64
 4   net_SC                         19590 non-null  float64
 5   time_first_sc_to_first_net_fu  19590 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 918.4 KB


In [188]:
# requestid duplicates? --> no, it was index anyways
# Show which requestid values appear more than once
duplicate_requestids = counts_df[counts_df['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")

Duplicate requestid values: []


In [189]:
# also df2 has no duplicates also because we did prviously checked and handled
duplicate_requestids = df3[df3['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")

Duplicate requestid values: []


In [190]:
# add qualitative data
# now we have 17k 0 2.2 k 1. where did 300+ 1s went? maybe they were from requests that never had a sc, we dropped them previously
final_df = counts_df.merge(df3, on='requestid', how='left')


# Fill non-matching requestids with 0 for the target
final_df["grosscontractsigned"] = final_df["grosscontractsigned"].fillna(0)

# Check the result
print(f"Final dataset shape: {final_df.shape}")
print(f"Target variable distribution:")
print(final_df["grosscontractsigned"].value_counts())
print(f"\nColumns: {list(final_df.columns)}")

Final dataset shape: (19590, 14)
Target variable distribution:
grosscontractsigned
0.0    17365
1.0     2225
Name: count, dtype: int64

Columns: ['requestid', 'gross_FU', 'gross_SC', 'net_FU', 'net_SC', 'time_first_sc_to_first_net_fu', 'zipregion', 'evaluationtime', 'desiredinstallationend', 'electricitybill', 'heatingbill', 'grosscontractsigned', 'selfipa_done', 'aggregated']


In [191]:
duplicate_requestids = final_df[final_df['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")

Duplicate requestid values: []


In [192]:
final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19590 entries, 0 to 19589
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   requestid                      19590 non-null  int64  
 1   gross_FU                       19590 non-null  int64  
 2   gross_SC                       19590 non-null  int64  
 3   net_FU                         19590 non-null  float64
 4   net_SC                         19590 non-null  float64
 5   time_first_sc_to_first_net_fu  19590 non-null  float64
 6   zipregion                      18999 non-null  object 
 7   evaluationtime                 19346 non-null  object 
 8   desiredinstallationend         19347 non-null  object 
 9   electricitybill                13536 non-null  float64
 10  heatingbill                    6462 non-null   float64
 11  grosscontractsigned            19590 non-null  float64
 12  selfipa_done                   19590 non-null 

#### Merging with behaviour from leadtime might happen here so null handling and encoding is not repeated

#### Encoding and Null handling

In [193]:
# Majority of bills have -1, 1 and 0

# set wrong bill values to nan
# Replace specific values with NaN
final_df['electricitybill'] = final_df['electricitybill'].replace([0, 1, -1], pd.NA)
final_df['heatingbill'] = final_df['heatingbill'].replace([0, 1, -1], pd.NA)

Optional for bill handling 
def clean_billing_column(series, column_name):
    """Clean billing column: set negative values and extreme outliers to replacement value"""
    
    # Convert to numeric, coerce errors to NaN
    series_clean = pd.to_numeric(series, errors='coerce')
    
    # Define reasonable bounds (adjust these based on your domain knowledge)
    min_reasonable = 0  # Bills shouldn't be negative
    max_reasonable = 2000  # Adjust based on your market (e.g., €1000/month seems high)
    
    # Count issues for reporting
    negative_count = (series_clean < min_reasonable).sum()
    outlier_count = (series_clean > max_reasonable).sum()
    
    print(f"{column_name}:")
    print(f"  - Negative values: {negative_count}")
    print(f"  - Values > {max_reasonable}: {outlier_count}")
    print(f"  - Original NaN: {series.isna().sum()}")
    
    # Replace problematic values
    series_clean.loc[series_clean < min_reasonable] = np.nan  # or -1
    series_clean.loc[series_clean > max_reasonable] = np.nan  # or -1
    
    return series_clean

# Apply cleaning
final_df['heatingbill'] = clean_billing_column(final_df['heatingbill'], 'heatingbill')
final_df['electricitybill'] = clean_billing_column(final_df['electricitybill'], 'electricitybill')

In [194]:
# before encoding, mark the missingness inside the row;
missing_cols = ['zipregion', 'evaluationtime', 'desiredinstallationend', 'electricitybill', 'heatingbill', 'aggregated']

for col in missing_cols:
    final_df[f'{col}_missing'] = final_df[col].isnull().astype(int)

# Check the predictive power of missing indicators
print("Missing indicator correlations with target:")
missing_indicators = [f'{col}_missing' for col in missing_cols]
missing_corrs = final_df[missing_indicators + ["grosscontractsigned"]].corr()["grosscontractsigned"].drop("grosscontractsigned")
print(missing_corrs.sort_values())

Missing indicator correlations with target:
electricitybill_missing          -0.043034
zipregion_missing                -0.022690
evaluationtime_missing           -0.003936
desiredinstallationend_missing   -0.002325
heatingbill_missing               0.078719
aggregated_missing                     NaN
Name: grosscontractsigned, dtype: float64


#### Encoding

In [195]:
# encoding 
# desiredinstallationend
type_mapping = {
    # Original Italian values
    '3-4mesi': 'three_to_four_months', 
    '5+mesi': 'more_than_5_months',
    '1-2mesi': 'one_to_two_months',
    'Non lo so': 'dont_know',
    'short': np.nan,
    # Already mapped values (keep as-is)
    'dont_know': 'dont_know',
    'three_to_four_months': 'three_to_four_months',
    'one_to_two_months': 'one_to_two_months', 
    'more_than_5_months': 'more_than_5_months',
    # Handle string 'nan'
    'nan': np.nan
}

final_df['desiredinstallationend1'] = final_df['desiredinstallationend'].map(type_mapping)

In [196]:
# evaluationtime
type_mapping = {
    # Original Italian values
    '3-6 mesi': np.nan,
    '<3 mesi': np.nan,
    '>6 mesi': np.nan,
    # Already mapped English values
    'less_than_three_months': np.nan,
    'more_than_six_months': np.nan,
    # Other values that appear in your data
    'understand_need': 'understand_need',  # or map to np.nan if you don't want these
    'understand_purchase': 'understand_purchase',  # or map to np.nan if you don't want these
    'evaluation': 'evaluation',  # or map to np.nan if you don't want these
    'curious': 'curious',  # or map to np.nan if you don't want these
    # Handle string 'nan'
    'nan': np.nan
}

# Create grouped_type column
final_df['evaluationtime1'] = final_df['evaluationtime'].map(type_mapping)


In [197]:
# Replace the old columns directly
final_df['desiredinstallationend'] = final_df['desiredinstallationend1']
final_df['evaluationtime'] = final_df['evaluationtime1']

# Drop the temporary columns
final_df = final_df.drop(['desiredinstallationend1', 'evaluationtime1'], axis=1)

In [198]:
# handle nan before encoding
final_df['desiredinstallationend'] = final_df['desiredinstallationend'].fillna('missing')
final_df['evaluationtime'] = final_df['evaluationtime'].fillna('missing')


In [199]:
# Start with ordinal encoding for time-based features

# Ordinal encoding for desiredinstallationend (time-based order)
time_order = [ 'one_to_two_months', 'three_to_four_months', 'more_than_5_months', 'dont_know', 'missing']
final_df['desiredinstallationend_encoded'] = final_df['desiredinstallationend'].map({val: i for i, val in enumerate(time_order)})

##### Analysis to see how to handle marketing and region

In [200]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19590 entries, 0 to 19589
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   requestid                       19590 non-null  int64  
 1   gross_FU                        19590 non-null  int64  
 2   gross_SC                        19590 non-null  int64  
 3   net_FU                          19590 non-null  float64
 4   net_SC                          19590 non-null  float64
 5   time_first_sc_to_first_net_fu   19590 non-null  float64
 6   zipregion                       18999 non-null  object 
 7   evaluationtime                  19590 non-null  object 
 8   desiredinstallationend          19590 non-null  object 
 9   electricitybill                 12735 non-null  object 
 10  heatingbill                     4681 non-null   object 
 11  grosscontractsigned             19590 non-null  float64
 12  selfipa_done                    

In [201]:
# For each categorical column, see conversion rates
print("Conversion rates by category:\n")

for col in ['zipregion', 'aggregated']:  # replace with your actual column names
    conversion_by_cat = final_df.groupby(col)["grosscontractsigned"].agg(['count', 'sum', 'mean'])
    conversion_by_cat.columns = ['total_samples', 'conversions', 'conversion_rate']
    conversion_by_cat = conversion_by_cat.sort_values('conversion_rate', ascending=False)
    
    print(f"\n{col.upper()}:")
    print(conversion_by_cat)
    print(f"Overall variation: {conversion_by_cat['conversion_rate'].std():.4f}")

Conversion rates by category:


ZIPREGION:
                       total_samples  conversions  conversion_rate
zipregion                                                         
Trentino-Alto Adige              170         40.0         0.235294
Friuli-Venezia Giulia           1157        236.0         0.203976
Liguria                          507         93.0         0.183432
Lombardia                       3150        435.0         0.138095
Piemonte                        1767        243.0         0.137521
Umbria                           285         37.0         0.129825
Molise                           114         14.0         0.122807
Veneto                          1488        181.0         0.121640
Valle D'Aosta                     75          9.0         0.120000
Toscana                         1333        157.0         0.117779
Emilia-Romagna                  1685        197.0         0.116914
Marche                           383         40.0         0.104439
Lazio              

Marketing gave good variance, would like to maintain that information. Region is not that significant but still good. 
However given the * of unique values, in both columns I will opt for grouping instead of each value having its column. 

Instead of 30+ categorical features, you get ~6-8, keeping the predictive power but losing the noise.

Downside; this grouping should occasionally double checked to see if it still makes sense

##### one hot encoding for marketing and region

In [202]:
# Define performance groups
aggregated_groups = {
    'High': ['Affiliation', 'Taboola'],           # >25% conversion
    'Medium': ['Google', 'Criteo', 'Organic'],    # 15-20% conversion  
    'Low': ['Outbrain', 'Meta', 'Other', 'MediaGo', 'Tik Tok']  # <12% conversion
}
def group_aggregated(value):
    if value == 'SEO':  # Handle the unreliable outlier
        return 'Medium'  # Conservative assignment
    
    for group, channels in aggregated_groups.items():
        if value in channels:
            return group
    return 'Low'  # fallback

# Apply grouping
final_df['mktg_grouped'] = final_df['aggregated'].apply(group_aggregated)

In [203]:
# Group regions by performance
def group_regions(region):
    # Top performers with good sample sizes
    if region in ['Trentino-Alto Adige', 'Friuli-Venezia Giulia', 'Liguria']:
        return 'High_Performer'  # 18-24% conversion
    
    # Large regions with solid performance  
    elif region in ['Lombardia', 'Piemonte', 'Veneto', 'Toscana', 'Emilia-Romagna']:
        return 'Large_Solid'     # 11-14% conversion
    
    # Smaller regions with decent samples
    elif region in ['Umbria', 'Lazio', 'Marche']:
        return 'Medium'          # 10-13% conversion
    
    # Lower performing regions
    else:
        return 'Lower'   

final_df['region_grouped'] = final_df['zipregion'].apply(group_regions)

In [204]:
# One-hot encode the grouped categories
final_df = pd.get_dummies(final_df, columns=['mktg_grouped', 'region_grouped'], prefix=['mktg', 'region'])

In [205]:
duplicate_requestids = final_df[final_df['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")


Duplicate requestid values: []


df is the log of all calendar_events so a direct join will make our dataset exponentially duplicated.

I will first get the unique requestid --> leadid cpairs and then merge that

In [206]:
requestid_leadid_pairs = df[['requestid', 'leadid']].drop_duplicates()

In [207]:
requestid_leadid_pairs[requestid_leadid_pairs['requestid'].duplicated(keep=False)]['requestid'].unique()

array([], dtype=int64)

In [208]:
# also add leadid to the final df for merging with behaviour data
# probably there is a cleaner way to do this

# Suppose the column you want from df1 is called "col_from_df1"
final_df = final_df.merge(
    requestid_leadid_pairs[['requestid', 'leadid']],  # keep only requestid + desired column
    on='requestid',                      # join key
    how='left'                           # keep all rows from df2
)


In [209]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19590 entries, 0 to 19589
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   requestid                       19590 non-null  int64  
 1   gross_FU                        19590 non-null  int64  
 2   gross_SC                        19590 non-null  int64  
 3   net_FU                          19590 non-null  float64
 4   net_SC                          19590 non-null  float64
 5   time_first_sc_to_first_net_fu   19590 non-null  float64
 6   zipregion                       18999 non-null  object 
 7   evaluationtime                  19590 non-null  object 
 8   desiredinstallationend          19590 non-null  object 
 9   electricitybill                 12735 non-null  object 
 10  heatingbill                     4681 non-null   object 
 11  grosscontractsigned             19590 non-null  float64
 12  selfipa_done                    

In [210]:
duplicate_requestids = final_df[final_df['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")

Duplicate requestid values: []


In [211]:
# any requestid with no leadid?
final_df[final_df['leadid'].isnull()]['requestid'].unique()

array([], dtype=int64)

In [212]:
# Drop original categorical columns and create final feature matrix
columns_to_drop = [ 'zipregion', 'evaluationtime', 'desiredinstallationend', 'aggregated']
final_df = final_df.drop(columns=columns_to_drop)


print(f"\nNo missing values: {final_df.isnull().sum().sum() == 0}")


No missing values: False


#### Imputing

In [213]:
# Fix electricitybill missing values
final_df['electricitybill'] = final_df['electricitybill'].fillna(final_df['electricitybill'].median())
final_df['heatingbill'] = final_df['heatingbill'].fillna(final_df['heatingbill'].median())

# Fix desiredinstallationend_encoded - NaN means 'Unknown' which should be 0 --> check if needed
#X_final['desiredinstallationend_encoded'] = X_final['desiredinstallationend_encoded'].fillna(0)

# Verify all missing values are gone
print(f"Remaining missing values: {final_df.isnull().sum().sum()}")
print(f"Dataset ready: {final_df.isnull().sum().sum() == 0}")



Remaining missing values: 0
Dataset ready: True


C:\Users\ElifYilmaz\AppData\Local\Temp\ipykernel_22316\1109642794.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df['electricitybill'] = final_df['electricitybill'].fillna(final_df['electricitybill'].median())
C:\Users\ElifYilmaz\AppData\Local\Temp\ipykernel_22316\1109642794.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df['heatingbill'] = final_df['heatingbill'].fillna(final_df['heatingbill'].median())


##### Scaling

In [214]:
# scaling when needed
continuous_cols= [
    'time_first_sc_to_first_net_fu',
    'electricitybill', 
    'heatingbill'
]

# Scale only the continuous features
scaler = StandardScaler()
final_df[continuous_cols] = scaler.fit_transform(final_df[continuous_cols])

In [215]:
behaviour.info()
# id here is leadid, dataset is preselected. 38k leads
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19981 entries, 0 to 19980
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          19981 non-null  int64  
 1   total_bc_attempts           19981 non-null  int64  
 2   total_bc_outcomes           19981 non-null  int64  
 3   lead_to_first_bc_days       19981 non-null  float64
 4   bc_duration_days            19981 non-null  float64
 5   bc_frequency                19981 non-null  float64
 6   positive_outcomes_count     19981 non-null  int64  
 7   negative_outcomes_count     19981 non-null  int64  
 8   noshow_outcomes_count       19981 non-null  int64  
 9   positive_outcome_ratio      19981 non-null  float64
 10  negative_outcome_ratio      19981 non-null  float64
 11  noshow_outcome_ratio        19981 non-null  float64
 12  reachability_score          19981 non-null  float64
 13  outcome_trend               199

Adding behavioral data from Booking Calls


In [216]:

merged_df = final_df.merge(behaviour, 
                          left_on='leadid', 
                          right_on='id', 
                          how='left')

In [217]:
merged_df.info()
# all matched correctly

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19590 entries, 0 to 19589
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   requestid                       19590 non-null  int64  
 1   gross_FU                        19590 non-null  int64  
 2   gross_SC                        19590 non-null  int64  
 3   net_FU                          19590 non-null  float64
 4   net_SC                          19590 non-null  float64
 5   time_first_sc_to_first_net_fu   19590 non-null  float64
 6   electricitybill                 19590 non-null  float64
 7   heatingbill                     19590 non-null  float64
 8   grosscontractsigned             19590 non-null  float64
 9   selfipa_done                    19590 non-null  int64  
 10  zipregion_missing               19590 non-null  int64  
 11  evaluationtime_missing          19590 non-null  int64  
 12  desiredinstallationend_missing  

all the new extra contracts i added are getting droped with no match in leadtime. make sure that they get grabbed correctly

In [218]:
duplicate_requestids = merged_df[merged_df['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")

# --> loooks ok 

Duplicate requestid values: []


#### Duplicate Check

In [219]:
# Check your actual dataset for duplicates
duplicate_cols = []
cols = merged_df.columns.tolist()

for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if merged_df.equals(merged_df[cols[j]]):
            duplicate_cols.append((cols[i], cols[j]))

print("Duplicate column pairs:", duplicate_cols)
print(f"Original shape: {merged_df.shape}")

# Remove duplicates if any found
X_final_clean = merged_df.loc[:, ~merged_df.columns.duplicated()]
print(f"After removing duplicates: {X_final_clean.shape}")

Duplicate column pairs: []
Original shape: (19590, 44)
After removing duplicates: (19590, 44)


In [220]:
# Check the original sizes
print("Original DataFrame sizes:")
print(f"final_df: {len(final_df)} rows")
print(f"behaviour: {len(behaviour)} rows")
print(f"merged_df: {len(merged_df)} rows")

print("\n" + "="*50)

# Check how many matches we got
matches = merged_df['id'].notna().sum()
no_matches = merged_df['id'].isna().sum()

print(f"Rows with matches: {matches} ({matches/len(merged_df)*100:.1f}%)")
print(f"Rows without matches: {no_matches} ({no_matches/len(merged_df)*100:.1f}%)")

print("\n" + "="*50)

# Check for duplicates in the key columns before merge
print("Checking for duplicates in join keys:")
print(f"Duplicates in final_df['leadid']: {final_df['leadid'].duplicated().sum()}")
print(f"Duplicates in final_df['requestid']: {final_df['requestid'].duplicated().sum()}")
print(f"Duplicates in behaviour['id']: {behaviour['id'].duplicated().sum()}")

print("\n" + "="*50)

# Check the overlap between the two key columns
final_leadids = set(final_df['leadid'])
behaviour_ids = set(behaviour['id'])

print(f"Unique leadids in final_df: {len(final_leadids)}")
print(f"Unique ids in behaviour: {len(behaviour_ids)}")
print(f"Common IDs: {len(final_leadids & behaviour_ids)}")
print(f"IDs only in final_df: {len(final_leadids - behaviour_ids)}")
print(f"IDs only in behaviour: {len(behaviour_ids - final_leadids)}")

Original DataFrame sizes:
final_df: 19590 rows
behaviour: 19981 rows
merged_df: 19590 rows

Rows with matches: 19590 (100.0%)
Rows without matches: 0 (0.0%)

Checking for duplicates in join keys:
Duplicates in final_df['leadid']: 4151
Duplicates in final_df['requestid']: 0
Duplicates in behaviour['id']: 0

Unique leadids in final_df: 15439
Unique ids in behaviour: 19981
Common IDs: 15439
IDs only in final_df: 0
IDs only in behaviour: 4542


In [221]:
# checking if they are converters or not
# Get the rows that didn't match (have NaN in the 'id' column from behaviour)
unmatched_rows = merged_df[merged_df['id'].isna()]

print(f"Analysis of {len(unmatched_rows)} unmatched rows:")
print("="*50)

# Check conversion rate for unmatched entries
conversion_rate_unmatched = unmatched_rows['grosscontractsigned'].mean()
total_conversions_unmatched = unmatched_rows['grosscontractsigned'].sum()

print(f"Unmatched entries conversion rate: {conversion_rate_unmatched:.3f} ({conversion_rate_unmatched*100:.1f}%)")
print(f"Total conversions among unmatched: {total_conversions_unmatched}")

print("\n" + "="*50)

# Compare with matched entries
matched_rows = merged_df[merged_df['id'].notna()]
conversion_rate_matched = matched_rows['grosscontractsigned'].mean()
total_conversions_matched = matched_rows['grosscontractsigned'].sum()

print(f"Matched entries conversion rate: {conversion_rate_matched:.3f} ({conversion_rate_matched*100:.1f}%)")
print(f"Total conversions among matched: {total_conversions_matched}")

print("\n" + "="*50)

# Overall comparison
print("COMPARISON:")
print(f"Unmatched: {conversion_rate_unmatched:.3f} ({conversion_rate_unmatched*100:.1f}%)")
print(f"Matched:   {conversion_rate_matched:.3f} ({conversion_rate_matched*100:.1f}%)")
print(f"Difference: {(conversion_rate_matched - conversion_rate_unmatched)*100:.1f} percentage points")

# Statistical significance test (optional)
from scipy import stats
if len(unmatched_rows) > 0 and len(matched_rows) > 0:
    stat, p_value = stats.chi2_contingency([[
        unmatched_rows['grosscontractsigned'].sum(), 
        len(unmatched_rows) - unmatched_rows['grosscontractsigned'].sum()
    ], [
        matched_rows['grosscontractsigned'].sum(), 
        len(matched_rows) - matched_rows['grosscontractsigned'].sum()
    ]])[:2]
    
    print(f"\nStatistical test p-value: {p_value:.6f}")
    print(f"Significant difference: {'Yes' if p_value < 0.05 else 'No'}")

Analysis of 0 unmatched rows:
Unmatched entries conversion rate: nan (nan%)
Total conversions among unmatched: 0.0

Matched entries conversion rate: 0.114 (11.4%)
Total conversions among matched: 2225.0

COMPARISON:
Unmatched: nan (nan%)
Matched:   0.114 (11.4%)
Difference: nan percentage points


In [222]:

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19590 entries, 0 to 19589
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   requestid                       19590 non-null  int64  
 1   gross_FU                        19590 non-null  int64  
 2   gross_SC                        19590 non-null  int64  
 3   net_FU                          19590 non-null  float64
 4   net_SC                          19590 non-null  float64
 5   time_first_sc_to_first_net_fu   19590 non-null  float64
 6   electricitybill                 19590 non-null  float64
 7   heatingbill                     19590 non-null  float64
 8   grosscontractsigned             19590 non-null  float64
 9   selfipa_done                    19590 non-null  int64  
 10  zipregion_missing               19590 non-null  int64  
 11  evaluationtime_missing          19590 non-null  int64  
 12  desiredinstallationend_missing  

In [223]:
# drop non converters with no behavioral data. keep the 7 that converted and recover their data manually
# Get the unmatched rows (the 860 entries without behavior data)
# Get the unmatched rows (the 860 entries without behavior data)
unmatched_rows = merged_df[merged_df['id'].isna()]

print(f"Original unmatched rows: {len(unmatched_rows)}")
print(f"Conversions among unmatched: {unmatched_rows['grosscontractsigned'].sum()}")
print(f"Non-conversions among unmatched: {(unmatched_rows['grosscontractsigned'] == 0).sum()}")

# Filter unmatched rows to keep only conversions (grosscontractsigned == 1)
unmatched_conversions = unmatched_rows[unmatched_rows['grosscontractsigned'] == 1]

print(f"\nAfter filtering unmatched rows:")
print(f"Keeping only unmatched conversions: {len(unmatched_conversions)}")

# Get the matched rows (the 13,296 entries with behavior data)
matched_rows = merged_df[merged_df['id'].notna()]

# Combine and replace merged_df
merged_df = pd.concat([matched_rows, unmatched_conversions], ignore_index=True)

print(f"\nFinal merged_df:")
print(f"Matched rows (with behavior): {len(matched_rows)}")
print(f"Unmatched conversions (without behavior): {len(unmatched_conversions)}")
print(f"Total rows: {len(merged_df)}")
print(f"All remaining unmatched rows are conversions: {(merged_df[merged_df['id'].isna()]['grosscontractsigned'] == 1).all()}")

Original unmatched rows: 0
Conversions among unmatched: 0.0
Non-conversions among unmatched: 0

After filtering unmatched rows:
Keeping only unmatched conversions: 0

Final merged_df:
Matched rows (with behavior): 19590
Unmatched conversions (without behavior): 0
Total rows: 19590
All remaining unmatched rows are conversions: True


In [224]:
# PROBLEM THREE
# see leads that didnt make it to sc --> le
merged_df.loc[merged_df['net_SC'] == 0, 'requestid']

9          1387
22         2039
57         3668
69         4202
72         4359
          ...  
19543    126967
19551    126980
19552    126981
19571    127538
19579    128268
Name: requestid, Length: 7310, dtype: int64

In [225]:
# out of 7k no net sc (but yess gross) 52 are converted could it be after the cutoff?
x = merged_df.loc[
    (merged_df['net_SC'] == 0) & (merged_df['grosscontractsigned'] == 1),
    'requestid'
]
x

9          1387
57         3668
69         4202
76         4502
90         4967
153        7227
190        9286
195        9600
246       12136
259       13082
294       14602
333       16694
338       16872
409       20096
456       22490
457       22538
477       23302
490       23673
498       24071
532       25455
565       26629
640       28791
642       28884
652       29149
671       29752
686       30361
714       31401
733       32048
735       32121
851       38499
903       40431
927       41579
942       42034
945       42088
955       42424
1013      46146
1024      46888
1067      48878
1073      49142
1143      53645
1350      66892
1739      88220
2158     106390
2159     106391
4694     109781
7581     113297
7683     113414
9411     115357
9628     115615
11966    118332
14474    121188
19005    126300
19571    127538
19579    128268
Name: requestid, dtype: int64

In [226]:
#to investigate
# ids
# 1387, 3668, 4202, 4502, 4967, 7227, 9286, 9600, 12136, 13082, 14602, 16694, 16872, 20096, 22490, 22538, 23302, 23673, 24071, 25455, 26629, 28791, 28884, 29149, 29752, 30361, 31401, 32048, 32121, 38499, 40431, 41579, 42034, 42088, 42424, 46146, 46888, 48878, 49142, 53645, 66892, 88220, 106390, 106391, 109781, 113297, 113414, 115357, 115615, 118332, 121188, 126300, 127538, 128268


they really dont have vd or vdfield that is not deleted --> sus entries. you can look into them later but for now lets drop only sc_net = 0 and grosscontractsigned = 1


In [227]:
# lets drop the ones who havent made to net sc
# Keep rows where not ( net_SC is equal to 0 and grosscontractsigned is 1)
merged_df = merged_df[~((merged_df['net_SC'] == 0) & (merged_df['grosscontractsigned'] == 1))]

In [228]:
# drop requestid ad other indicatiors
merged_df = merged_df.drop(columns=['requestid', 'id', 'leadid'])

In [229]:
# Check conversions in missing call data rows
missing_mask = merged_df['total_bc_attempts'].isnull()
print(f"Conversions in missing rows: {merged_df[missing_mask]['grosscontractsigned'].sum()}")
print(f"Conversion rate in missing rows: {merged_df[missing_mask]['grosscontractsigned'].mean():.4f}")
print(f"Conversion rate in complete rows: {merged_df[~missing_mask]['grosscontractsigned'].mean():.4f}")

Conversions in missing rows: 0.0
Conversion rate in missing rows: nan
Conversion rate in complete rows: 0.1111


In [230]:
# some of these requests are very old and dont have propre registstration, some have some registration but essentially there are only 7 contract signed insode 860, so essentially dropping as a first step of undersampling.
# ill keep 7 positives and recover their data, drop the rest

In [231]:
# this would drop all entries with any missing values, so lets hold our horses here. 
#merged_df = merged_df.dropna().copy()
print(f"New dataset shape: {merged_df.shape}")
print(f"New conversion rate: {merged_df['grosscontractsigned'].mean():.4f}")
print(f"Total conversions kept: {merged_df['grosscontractsigned'].sum()}")

New dataset shape: (19536, 41)
New conversion rate: 0.1111
Total conversions kept: 2171.0


In [232]:
# Create subfolder if it doesn't exist
os.makedirs('processed_data', exist_ok=True)

# Save your merged dataframe
merged_df.to_csv('processed_data/merged_df.csv', index=False)

print("Dataset saved successfully!")

Dataset saved successfully!


In [233]:
merged_df["grosscontractsigned"].value_counts() 

grosscontractsigned
0.0    17365
1.0     2171
Name: count, dtype: int64